In [2]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.0 MB/s eta 0:00:00


In [3]:
import tensorflow as tf

In [4]:
from tensorflow import keras
import numpy as np

In [5]:
print(tf.__version__)

2.14.0


In [6]:
fashion_mnist = keras.datasets.fashion_mnist

In [7]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [8]:
train_images = train_images/255.0
test_images = test_images/255.0


In [9]:
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [10]:
train_images.shape #this is 28 , 28 pixels

(60000, 28, 28)

In [11]:
#reshape the images
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [12]:
#create a function called build model
#hp is hyperparameter
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters=hp.Int('conv_1_filter',min_value=32,max_value=128, step=16),
          kernel_size=hp.Choice('conv_1_kernel',values = [3,5]),
          activation = 'relu',
          input_shape=(28,28,1)
        ),
      keras.layers.Conv2D(
          filters = hp.Int('conv_2_filter',min_value=32, max_value =64,step=16),
          kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
          activation = 'relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
          activation='relu'
      ),
      keras.layers.Dense(10,activation='softmax')
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model


In [13]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-13-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [14]:
tuner_search = RandomSearch(build_model,objective = 'val_accuracy',max_trials=5,directory='output', project_name = 'Mnist Fashion')

In [15]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1) #the object will try to search best parameter

Trial 5 Complete [00h 00m 30s]
val_accuracy: 0.9133333563804626

Best val_accuracy So Far: 0.9138333201408386
Total elapsed time: 00h 03m 09s


In [17]:
model=tuner_search.get_best_models(num_models=1)[0]

In [19]:
model.summary()  #here we get the best model, and after that we retrain the model with these best parameters

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 128)       1280      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 48)        153648    
                                                                 
 flatten (Flatten)           (None, 23232)             0         
                                                                 
 dense (Dense)               (None, 112)               2602096   
                                                                 
 dense_1 (Dense)             (None, 10)                1130      
                                                                 
Total params: 2758154 (10.52 MB)
Trainable params: 2758154 (10.52 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 12s 6ms/step - loss: 0.1301 - accuracy: 0.9513 - val_loss: 0.2742 - val_accuracy: 0.9097
Epoch 5/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0900 - accuracy: 0.9666 - val_loss: 0.3010 - val_accuracy: 0.9172
Epoch 6/10
1688/1688 [==============================] - 11s 7ms/step - loss: 0.0651 - accuracy: 0.9757 - val_loss: 0.3197 - val_accuracy: 0.9105
Epoch 7/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0471 - accuracy: 0.9823 - val_loss: 0.4294 - val_accuracy: 0.9125
Epoch 8/10
1688/1688 [==============================] - 11s 7ms/step - loss: 0.0350 - accuracy: 0.9874 - val_loss: 0.4192 - val_accuracy: 0.9132
Epoch 9/10
1688/1688 [==============================] - 11s 7ms/step - loss: 0.0284 - accuracy: 0.9903 - val_loss: 0.5136 - val_accuracy: 0.9138
Epoch 10/10
1688/1688 [==============================] - 11s 7ms/step - loss: 0.0242 - accuracy: 0.9914 - val_loss: 0.5775 - val_a